In [1]:
import pandas as pd 
import numpy as np
from pandas import DataFrame,Series

## 5.2.1 merge合并

### merge函数是通过一个或多个键（DataFrame的列）将两个DataFrame按行合并起来，其方式与关系型数据库一样。

In [2]:
price = DataFrame({
    'fruit':['apple','banana','orange'],
    'price':[23,32,45]
})
amount = DataFrame({
    'fruit':['apple','banana','apple','apple','banana','pear'],
    'amount':[5,3,6,3,6,7]
})

In [3]:
price

,fruit,price
0,apple,23
1,banana,32
2,orange,45


In [4]:
amount

,fruit,amount
0,apple,5
1,banana,3
2,apple,6
3,apple,3
4,banana,6
5,pear,7


### 这是多对一的合并情况

In [5]:
pd.merge(amount,price)

,fruit,amount,price
0,apple,5,23
1,apple,6,23
2,apple,3,23
3,banana,3,32
4,banana,6,32


### 由于两个DataFrame都有fruit列名，所以默认按该列进行合并。当然，也可以指定键名，如果两个DataFrame的列名不一样，也可以单独指定

In [6]:
pd.merge(price,amount,on='fruit')

,fruit,price,amount
0,apple,23,5
1,apple,23,6
2,apple,23,3
3,banana,32,3
4,banana,32,6


In [7]:
pd.merge(amount,price,left_on='fruit',right_on='fruit')

,fruit,amount,price
0,apple,5,23
1,apple,6,23
2,apple,3,23
3,banana,3,32
4,banana,6,32


### merge默认为内连接（inner），也就是返回交集。通过how参数可以选择连接方法：左连接（left）、右连接（right）和外连接（outer）

In [8]:
pd.merge(amount,price,how='left')

,fruit,amount,price
0,apple,5,23.0
1,banana,3,32.0
2,apple,6,23.0
3,apple,3,23.0
4,banana,6,32.0
5,pear,7,NaN


In [9]:
pd.merge(amount,price,how='right')

,fruit,amount,price
0,apple,5.0,23
1,apple,6.0,23
2,apple,3.0,23
3,banana,3.0,32
4,banana,6.0,32
5,orange,NaN,45


In [10]:
pd.merge(amount,price,how='outer')

,fruit,amount,price
0,apple,5.0,23.0
1,apple,6.0,23.0
2,apple,3.0,23.0
3,banana,3.0,32.0
4,banana,6.0,32.0
5,pear,7.0,NaN
6,orange,NaN,45.0


### 多对多的连接会产生笛卡尔积。左边的DataFrame有3个apple，右边有2个apple，这样连接的DataFrame就有6个apple

In [11]:
price2 = DataFrame({
    'fruit':['apple','banana','orange','apple'],
    'price':[23,32,45,25]
})
amount2 = DataFrame({
    'fruit':['apple','banana','apple','apple','banana','pear'],
    'amount':[5,3,6,3,6,7]
})

In [12]:
price2

,fruit,price
0,apple,23
1,banana,32
2,orange,45
3,apple,25


In [13]:
amount2

,fruit,amount
0,apple,5
1,banana,3
2,apple,6
3,apple,3
4,banana,6
5,pear,7


In [14]:
pd.merge(price2,amount2)

,fruit,price,amount
0,apple,23,5
1,apple,23,6
2,apple,23,3
3,apple,25,5
4,apple,25,6
5,apple,25,3
6,banana,32,3
7,banana,32,6


In [17]:
left = pd.DataFrame({
    'key1':['one','one','two'],
    'key2':['a','b','a'],
    'val1':[2,3,4]
})
right = pd.DataFrame({
    'key1':['one','one','two','two'],
    'key2':['a','a','a','b'],
    'val2':[5,6,7,8]
})

In [18]:
left

,key1,key2,val1
0,one,a,2
1,one,b,3
2,two,a,4


In [19]:
right

,key1,key2,val2
0,one,a,5
1,one,a,6
2,two,a,7
3,two,b,8


In [20]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,val1,val2
0,one,a,2.0,5.0
1,one,a,2.0,6.0
2,one,b,3.0,NaN
3,two,a,4.0,7.0
4,two,b,NaN,8.0


### 在合并时要考虑到重复列名的问题，虽然可以人为进行重复列名的修改，但merge函数提供了suffixes用于处理该问题。

In [21]:
pd.merge(left,right,on='key1')

,key1,key2_x,val1,key2_y,val2
0,one,a,2,a,5
1,one,a,2,a,6
2,one,b,3,a,5
3,one,b,3,a,6
4,two,a,4,a,7
5,two,a,4,b,8


In [22]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,val1,key2_right,val2
0,one,a,2,a,5
1,one,a,2,a,6
2,one,b,3,a,5
3,one,b,3,a,6
4,two,a,4,a,7
5,two,a,4,b,8


### 有时连接的键位于DataFrame的行索引上，可通过传入left_index=True或者right_index=True指定将索引作为连接键来使用

In [6]:
import pandas as pd 
import numpy as np
from pandas import DataFrame,Series

In [11]:
left2 = pd.DataFrame({
    'key':['a','a','b','b','c'],
    'val1':[0,1,2,3,4]
})
right2 = pd.DataFrame({
    'val2':[5,7]},index=['a','b']
)

In [12]:
left2

,key,val1
0,a,0
1,a,1
2,b,2
3,b,3
4,c,4


In [9]:
right2

,val2
a,5
b,7


In [13]:
pd.merge(left2,right2,left_on='key',right_index=True)

,key,val1,val2
0,a,0,5
1,a,1,5
2,b,2,7
3,b,3,7


### DataFrame中有一个join方法，可以快速完成按索引合并

In [14]:
left3 = pd.DataFrame({
    'val1':[0,1,2,3]
},index=['a','b','a','c'])
right3 = pd.DataFrame({
    'val2':[5,7]},index=['a','b']
)

In [15]:
left3.join(right3,how='outer')

,val1,val2
a,0,5.0
a,2,5.0
b,1,7.0
c,3,NaN


## 5.2.2 concat连接

### 如果需要合并的DataFrame之间没有连接键，就不能使用merge方法了，这时可通过pandas的concat方法实现。

In [19]:
s1 = Series([0,1],index=['a','b'])
s2 = Series([2,3],index=['c','d'])
s3 = Series([4,5],index=['e','f'])

In [20]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
dtype: int64

### 默认情况下，concat是在axis=0上工作的，当然通过指定轴向也可以按列进行连接

In [21]:
pd.concat([s1,s2,s3],axis=1)

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,NaN,4.0
f,NaN,NaN,5.0


### 这样就会生成一个DataFrame。通过结果可以看出，这种连接方式为外连接（并集），通过传入join='inner’可以实现内连接

### 可以通过join_axes指定使用的索引顺序

In [23]:
s4 = pd.concat([s1*10,s3])
s4

a     0
b    10
e     4
f     5
dtype: int64

In [24]:
pd.concat([s1,s4],axis=1)

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,10
e,NaN,4
f,NaN,5


In [25]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,10


In [26]:
pd.concat([s1,s4],axis=1,join_axes=[['b','a']],join='inner')

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,0,1
b,1,10
a,0,0


### 注意：concat只有内连接和外连接。

### 参与连接的数据对象在结果中是分不开的，可通过keys参数给连接对象创建一个层次化索引

In [27]:
pd.concat([s1,s4])

a     0
b     1
a     0
b    10
e     4
f     5
dtype: int64

In [28]:
pd.concat([s1,s4],keys=['one','two'])

one  a     0
     b     1
two  a     0
     b    10
     e     4
     f     5
dtype: int64

### 如果按列连接，keys就成了DataFrame的列索引。concat连接对于DataFrame是同样适用的

In [31]:
df1 = pd.DataFrame({
    'val1':[0,1,2]
},index=['a','b','c'])
df2 = pd.DataFrame({
    'val2':[5,7]},index=['a','b']
)

In [32]:
df1

,val1
a,0
b,1
c,2


In [34]:
df2

,val2
a,5
b,7


In [35]:
pd.concat([df1,df2],axis=1,keys=['one','two'])

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two
,val1,val2
a,0,5.0
b,1,7.0
c,2,NaN


### 除了传入列表，通过字典数据也可以完成连接，字典的键就是keys的值

In [36]:
pd.concat({'one':df1,'two':df2},axis=1)

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two
,val1,val2
a,0,5.0
b,1,7.0
c,2,NaN


### 当行索引类似时，通过默认连接会出现重复行索引。这时可通过ignore_index=‘True’忽略索引，以达到重排索引的效果

In [39]:
df1 = DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df2 = DataFrame(np.random.randn(2,2),columns=['d','c'])

In [40]:
df1

,a,b,c,d
0,-0.224826,-1.429490,1.420865,-0.696664
1,0.364856,-1.146335,-1.373627,-0.354697
2,0.022402,0.486444,0.003036,2.045794


In [41]:
df2

,d,c
0,2.386894,0.193605
1,0.225810,1.252335


In [42]:
pd.concat([df1,df2])

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.224826,-1.429490,1.420865,-0.696664
1,0.364856,-1.146335,-1.373627,-0.354697
2,0.022402,0.486444,0.003036,2.045794
0,NaN,NaN,0.193605,2.386894
1,NaN,NaN,1.252335,0.225810


In [43]:
pd.concat([df1,df2],ignore_index=True)

C:\Users\aaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.224826,-1.429490,1.420865,-0.696664
1,0.364856,-1.146335,-1.373627,-0.354697
2,0.022402,0.486444,0.003036,2.045794
3,NaN,NaN,0.193605,2.386894
4,NaN,NaN,1.252335,0.225810


## 5.2.3 combine_first合并

### 如果需要合并的两个DataFrame存在重复的索引，在这种情况下，若使用merge和concat方法都不能准确地解决问题，此时就需要使用combine_first方法，该方法类似于“打补丁”

In [44]:
df1 = DataFrame({'a':[3.0,np.nan,6.0,np.nan],
                'b':[np.nan,4.0,6.0,np.nan]})
df1

,a,b
0,3.0,NaN
1,NaN,4.0
2,6.0,6.0
3,NaN,NaN


In [45]:
df2 = DataFrame({'a':range(5),'b':range(5)})
df2

,a,b
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [46]:
df1.combine_first(df2)

,a,b
0,3.0,0.0
1,1.0,4.0
2,6.0,6.0
3,3.0,3.0
4,4.0,4.0


## 5.2.4 数据重塑

### 数据重塑用于重排DataFrame，有两个常用的方法：stack方法用于将DataFrame的列“旋转”为行；unstack方法用于将DataFrame的行“旋转”为列。

In [47]:
df = DataFrame(np.arange(9).reshape(3,3),
              index=['a','b','c'],
              columns=['one','two','three'])
df.index.name = 'alph'
df.columns.name = 'number'
df

number,one,two,three
alph,,,
a,0,1,2
b,3,4,5
c,6,7,8


In [49]:
result = df.stack()
result

alph  number
a     one       0
      two       1
      three     2
b     one       3
      two       4
      three     5
c     one       6
      two       7
      three     8
dtype: int32

### 将列转换为行后，则生成了一个Series数据，通过unstack又会将其重排为原始数据的形式

In [50]:
result.unstack()

number,one,two,three
alph,,,
a,0,1,2
b,3,4,5
c,6,7,8


### 默认情况下，数据重塑的操作都是最内层的，也可以通过级别编号或名称来指定其他级别进行重塑操作

In [51]:
result.unstack(0)

alph,a,b,c
number,,,
one,0,3,6
two,1,4,7
three,2,5,8


In [52]:
result.unstack('alph')

alph,a,b,c
number,,,
one,0,3,6
two,1,4,7
three,2,5,8


### 不仅数据重塑的操作是最内层的，操作的结果也会使旋转轴位于最低级别

In [56]:
df = DataFrame(np.arange(16).reshape(4,4),
              index=[['one','one','two','two'],['a','b','a','b']],
              columns=[['apple','apple','orange','orange'],['red','green','red','green']])
df

apple       orange      
        red green    red green
one a     0     1      2     3
    b     4     5      6     7
two a     8     9     10    11
    b    12    13     14    15

In [57]:
df.stack()

apple  orange
one a green      1       3
      red        0       2
    b green      5       7
      red        4       6
two a green      9      11
      red        8      10
    b green     13      15
      red       12      14

In [58]:
df.unstack()

apple               orange              
      red     green        red     green    
        a   b     a   b      a   b     a   b
one     0   4     1   5      2   6     3   7
two     8  12     9  13     10  14    11  15